<a href="https://www.kaggle.com/code/shravankumar147/03-hf-chain-parser?scriptVersionId=214964499" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# First, install required packages:
!pip install langchain langchain-community langchain-core langchain_huggingface transformers torch pypdf accelerate --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 111.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 

In [2]:
from huggingface_hub import notebook_login

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from langchain_community.llms import HuggingFacePipeline
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate

In [3]:
notebook_login()

# Kaggle Approach to login
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("HUGGINGFACE_TOKEN")


In [4]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
task = "text-generation"

pipe = pipeline(
    task,
    model=model,
    tokenizer=tokenizer,
    max_length=1024,
    temperature=0.3,
    top_p=0.85,
    repetition_penalty=1.1,
    do_sample=True,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id
)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
llm = HuggingFacePipeline(pipeline=pipe)

In [7]:
system_prompt = """You are tasked with parsing a resume. Your objective is to extract relevant information in a valid structured 'JSON' format. Do not write explanations or any other preambles, do not add or write anything out of context, use only the given information"""

human_prompt = """
             **Task:** Extract key information from the following resume text.

            **Resume Text:**
            {context}

            **Instructions:**
            Please extract the following information and format it in a clear structure:

            1. **Contact Information:**
            - Name:
            - Email:
            - Phone Number:
            - Website/Portfolio/LinkedIn:
            - Github Profile:

            2. **Education:**
            - Institution Name:
            - Degree:
            - Field of Study:
            - Graduation Date:

            3. **Experience:**
            - Job Title:
            - Company Name:
            - Location:
            - Dates of Employment:
            - Responsibilities/Projects:

            4. **Projects:**
            - Project Title:
            - Description/Technologies Used:
            - Outcomes/Results:

            5. **Skills:**
            - Programming Languages:
            - Technologies/Tools/frameworks:

            6. **Additional Information:** (if applicable)
            - Certifications:
            - Awards or Honors:
            - Professional Affiliations:
            - Languages:

            **Question:**
            {question}

            **Extracted Information:**
        """

In [8]:
context = """Bharath Kumar Parunandula\n+ Hyderabad\n# bharathkumar1011@gmail.com\n\x84 8639078566\nð bharathkumar parunandula\n§ Bharathkumar1011\nWelcome\nTo secure an entry-level position as a Junior Data Analyst or Data Scientist, where I can apply my skills in data analysis\nand visualization to contribute to organizational success while continuously developing my expertise. .\nPROJECTS\n• House Prices - Advanced Regression Techniques\n• Titanic - Machine Learning from Disaster\n• Iris dataset\n• Air Quality Index\nCirtificate\n• Programming for Everybody\n• Using Databases with Python\nEducation\nB.tech\nInstitute of Aeronautical Engineering,Aeronautical Engineering\nGPA: 6.18/10\n2014 - 2018\nSkills\n• Programming Languages: Python, SQLite\n• Data Analysis: Statistical analysis, data cleaning, and data visualization\n• Tools and Software: Excel, VS code\n• Machine Learning: Linear regression, decision trees, and clustering\n• Database Management: SQLite\n• Technical Skills: ETL processes, Exploratory Data Analysis (EDA)\n• Soft Skills: Deep Thinking, teamwork, communication, and attention to detail\nBooks\n• The Alchemist\nbharath kumar parunandula - Page 1 of 1\nLast updated in September 2024\n"""
question = "Extract key information from this resume."
template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", human_prompt),
    ]
)

complete_prompt = template.format_messages(context=context, question=question)

In [9]:
print(complete_prompt)

[SystemMessage(content="You are tasked with parsing a resume. Your objective is to extract relevant information in a valid structured 'JSON' format. Do not write explanations or any other preambles, do not add or write anything out of context, use only the given information", additional_kwargs={}, response_metadata={}), HumanMessage(content='\n             **Task:** Extract key information from the following resume text.\n\n            **Resume Text:**\n            Bharath Kumar Parunandula\n+ Hyderabad\n# bharathkumar1011@gmail.com\n\x84 8639078566\nð bharathkumar parunandula\n§ Bharathkumar1011\nWelcome\nTo secure an entry-level position as a Junior Data Analyst or Data Scientist, where I can apply my skills in data analysis\nand visualization to contribute to organizational success while continuously developing my expertise. .\nPROJECTS\n• House Prices - Advanced Regression Techniques\n• Titanic - Machine Learning from Disaster\n• Iris dataset\n• Air Quality Index\nCirtificate\n• Pr

In [10]:
%%time
response = llm.invoke(complete_prompt, skip_prompt=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


CPU times: user 15.6 s, sys: 84.3 ms, total: 15.6 s
Wall time: 16.9 s


In [11]:
print(response)

 {
          "Name": "Bharath Kumar Parunandula",
          "Contact Information": {
            "Email": "+ bharathkumar1011@gmail.com",
            "Phone Number": "8639078566",
            "Website/Portfolio/LinkedIn": "",
            "Github Profile": ""
          },
          "Education": {
            "Institution Name": "Institute of Aeronautical Engineering",
            "Degree": "B.tech",
            "Field of Study": "Aeronautical Engineering",
            "Graduation Date": "2014-2018"
          },
          "Projects": [
            {
              "Project Title": "House Prices - Advanced Regression Techniques",
              "Description/Technologies Used": "Advanced Regression Techniques",
              "Outcomes/Results": null
            },
            {
              "Project Title": "Titanic - Machine Learning from Disaster",
              "Description/Technologies Used": "Machine Learning from Disaster",
              "Outcomes/Results": null
            },
       